<a href="https://colab.research.google.com/github/youse0ng/pytorch_practice/blob/main/03_pytorch_computer_vision_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Pytorch Computer Vision

* See reference online book - https://www.learnpytorch.io/03_pytorch_computer_vision/

* https://github.com/mrdbourke/pytorch-deep-learning/blob/main/helper_functions.py


## 0. Computer vision libraries in Pytorch

* `torchvision` - base domain library for Pytorch computer vision
* `torchvision.datasets` - get datasets and data loading functions for computuer vision here
* `torchvision.models` - get pretrained computer vision models that you can leverage for your own problems
* `torchvision.transform` - functions for manipulating your vision data (images) to be suitable foruse with an ML model
* `torch.utils.data.Dataset` - Base dataset class for Pytorch.
* `torch.utils.data.DataLoader` - Creates a Python iterable over a dataset

In [ ]:
# Import Pytorch
import torch
from torch import nn

# Import torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

# Import matplotlib for visualization
import matplotlib.pyplot as plt

# Check versions
print(torch.__version__)
print(torchvision.__version__)

## 1. Getting a dataset

Fashion Mnist : The dataset we'll be using from torchvision.datasets

In [ ]:
# Setup Training data
from torchvision import datasets
train_data=datasets.FashionMNIST(
    root="data ", # where to download data to?
    train=True, # 학습 데이터가 필요한가?
    download=True, # 다운로드 할까요?
    transform=torchvision.transforms.ToTensor(), # 어떻게 데이터를 변환하고 싶은가요?
    #ToTensor(): Convert a PIL(Python imaging Library) Image or numpy.ndarray to Tensor
    target_transform=None, # Do we want to transform the label? (No) 어떻게 레이블을 변환하고 싶은가요?
)
test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=None,
)

`ToTensor()`는

Convert PIL Image or numpy.ndarray(H x W x C) (height width Channel)의 (0,255) 픽셀값들을 (C H W)로 범위는 0~ 1 사이로 변환

In [ ]:
len(train_data),len(test_data),type(train_data),type(test_data)

In [ ]:
# See the first training example
image,label=train_data[0] # 튜플 형식
image,label

In [ ]:
# 클래스 확인 방법 1
class_names=train_data.classes
class_names

In [ ]:
# 클래스 확인방법 2
class_to_idx=train_data.class_to_idx
class_to_idx

In [ ]:
train_data.targets

In [ ]:
# Check the shape of our image
print(f"image.shape: {image.shape}, [Color_Channel, height, width]")
print(f"image label:{class_names[label]}")

### 1.2 Visualizing our data

In [ ]:
import matplotlib.pyplot as plt
image,label=train_data[0]
print(f"Image shape: {image.shape}")
plt.imshow(image.squeeze())
plt.title(label)
# 매트플롯립은 높이와 너비만을 예상하므로 [28,28]로 바꾸어줘야한다. 단일 차원 제거.

In [ ]:
plt.imshow(image.squeeze(),cmap="gray")
plt.title(class_names[label])
#plt.axis(False) 축 삭제.

In [ ]:
# Plot more images
torch.manual_seed(42)
fig=plt.figure(figsize=(8,8))
rows, cols=4,4
for i in range(1,rows*cols+1):
  random_idx=torch.randint(0,len(train_data),size=[1]).item()
  img,label=train_data[random_idx]
  fig.add_subplot(rows,cols,i)
  plt.imshow(img.squeeze(),cmap="gray")
  plt.title(class_names[label])
  plt.axis(False)


Do you think these items of clothing coulde be modelled with pure linear lines or do you think we'll need non-linearities

- 너가 이 데이터 봤을때 선형식이 필요한거같아 아닌거 같아?

## 2. Prepare Dataloader

Right now, our data is in the form of Pytorch Datasets
지금 현재 데이터는 Pytorch 데이터 세트 형식이다.

`DataLoader` turns our dataset into a Python iterable

More specifically, we want to turn our data into batches (or mini-batches).

Why would we do this?
28 * 28 의 샘플 60000개를 한번에 볼 수 있다는건 메모리가 그만큼 상당히 필요해야한다.

배치 사이즈와 미니배치를 하는 이유
1. It is more computationally efficient, as in, your computing hardware may not be able to look (store in memory) at 60000 images in one hit. So we break it down to 32 images at a time (batch size of 32).
2. It gives our neural network more chances to update its gradients per epoch.

In [ ]:
# 데이터 형식
train_data,test_data

In [ ]:
from torch.utils.data import DataLoader

# Setup the batch size hyperparameter
BATCH_SIZE=32

# Turn datasets into iterables (batches)
train_dataloader=DataLoader(dataset=train_data,
                            batch_size=BATCH_SIZE,
                            shuffle=True)

test_dataloader=DataLoader(dataset=test_data,
                           batch_size=BATCH_SIZE,
                           shuffle=False) #테스트 데이터셋은 셔플을 안 시키는 것이 오히려 더 평가할때 쉽다.

train_dataloader,test_dataloader

In [ ]:
# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}")
print(f"Length of train_dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}...")
print(f"Length of test_dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}...")

In [ ]:
# Check out what's inside? the training dataloader
train_features_batch, train_labels_batch=next(iter(train_dataloader)) # 안의 내용을 확인하기 위한 iter함수 사용 (32개의 텐서가 들어가있는 1batch)
train_features_batch.shape, train_labels_batch.shape

In [ ]:
# Show a sample
#torch.manual_seed(42)
random_idx=torch.randint(0,len(train_features_batch),size=[1]).item()
img,label=train_features_batch[random_idx],train_labels_batch[random_idx]
plt.imshow(img.squeeze(),cmap="gray")
plt.title(class_names[label])
print(f"Image size: {img.shape}")
print(f"Label:{label},label_size={label.shape}")

## 3. Model 0: Build a baseline model

When starting to build a series of machine learning modelling experiments, it's best practice to start with a baseline model
기계학습 모델링을 세우기 시작할때, 가장 좋은 연습은 기본 모델부터 시작하는게 좋다.

A baseline model is a simple model you will try and improve upon with subsequent models/experiments

In other words: Start simply and add complexity when necessary

간단하게 시작해서 복잡성을 추가하라 필요할 때



In [ ]:
# Create a flatten layer
flatten_model=nn.Flatten()

# Get a single sample
x=train_features_batch[0]
x.shape

# Flatten the sample
output=flatten_model(x) # perform forward pass

# Print out what happened
print(f"shape before flattening :{x.shape} -> [color_channel,height,width]")
print(f"shape after flattening:{output.shape} -> [color_channel,height * width]")

In [ ]:
from torch import nn
class FashionMNISTModelV0(nn.Module):
  def __init__(self,
               input_shape:int,
               hidden_units:int,
               output_shape:int):
    super().__init__()
    self.layer_stack=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape,
                  out_features=hidden_units),
        nn.Linear(in_features=hidden_units,
                  out_features=output_shape)
    )
  def forward(self,x):
    return self.layer_stack(x)

In [ ]:
torch.manual_seed(42)

# Setup model with input parameters
model_0=FashionMNISTModelV0(
    input_shape=784, #this is 28*28
    hidden_units=10,
    output_shape=len(class_names) # one for every class
)
model_0.to("cpu")

In [ ]:
dummy_x=torch.rand([1,1,28,28])
model_0(dummy_x)

### 3.1 Setup loss, optimizer and evaluation metrics

* Loss function - Since we're working with multi-class data, our loss function will be `nn.CrossEntropyLoss()`
* Optimizer - our Optimizer `torch.optim.SGD()`
* Evaluation metric - Since we're working on a classification problem , let's use accuracy as our evaluation metric.

In [ ]:
# Calculate accuracy (classfication metric)
def accuracy_fn(y_true,y_pred):
  correct=torch.eq(y_true,y_pred).sum().item()
  acc=(correct/len(y_pred)) * 100
  return acc

In [ ]:
import requests
from pathlib import Path

# Download helper functions from Learn Pytorch Repo
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download...")
else:
  print("Downloading helper_function.py")
  request=requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py","wb") as f:
    f.write(request.content)

In [ ]:
# Import accuracy metric
from helper_functions import accuracy_fn

# Setup loss function and optimizer
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(params=model_0.parameters(),
                          lr=0.1)

### 3.2 Creating a function to time our experiments

Machine learning is very experimental.

Two of the main things you'll often want to track are:
1. Model's performance (loss and accuracy values etc)
2. How fast it runs

In [ ]:
from timeit import default_timer as timer
def print_train_time(start:float,
                     end: float,
                     device:torch.device=None):
  """Prints difference between start and end time."""
  total_time=end-start
  print(f"Train time on {device}: {total_time:.3f} seconds")
  return total_time

In [ ]:
start_time=timer()
# some code...
end_time=timer()
print_train_time(start_time,end=end_time,device="cpu")

### 3.3 Creating a traning loop and training a model on batches of data

Highlight that the optimizer will update a model's parameters once per batch rather than once per epoch...

#배치마다 모델의 파라미터를 업데이트해준다는 점을 강조한다.

1. Loop through epochs
2. Loop through training batches, perform training steps, calculate the train loss *per batch*.
3. Loop through testing batches, perform testing steps, calculate the test loss *per batch*.
4. print out what's happening.
5. Time it all (for fun).


In [ ]:
# Check out enumerate
for batch,(X,y) in enumerate(train_dataloader):
  pass

In [ ]:
for X_test,y_test in enumerate(test_dataloader):
  print(X_test)


In [ ]:
# Import tqdm for progress bar.
from tqdm.auto import tqdm

# Set the seed and start timer
torch.manual_seed(42)
train_time_start_on_cpu=timer()

# set the number of epochs (we'll keep this small for faster training time)
epochs=2

# 전체 epoch loop 안에 batch loop가 있는 형

# Create train and test loop
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n------")
  ### Training
  train_loss=0 # Calculate the train loss per batch --> accumulate
  # Add a loop to loop through the training batches
  for batch,(X,y) in enumerate(train_dataloader): # enumerate 원소랑 인덱싱 번호를 추출.
    model_0.train()
    # 1. Forward pass
    y_pred=model_0(X)
    print(f"batch={batch}")
    # 2. Calculate lsos(per batch)
    loss=loss_fn(y_pred,y)
    train_loss+=loss # accumulate train loss
    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. loss backward
    loss.backward() # 모델의 파라미터 기울기를 계산한다.

    # 5. Optimizer step
    optimizer.step() # 가중치 업데이트

    # Print out what's happening
    if batch % 400==0:
      print(f"Looked at {batch * len(X)}/ {len(train_dataloader.dataset)} samples") # batch : 1874 len(X):32

 # Divide total train loss by length of train dataloader
  train_loss /=len(train_dataloader)
  print(f"train_loss={train_loss}")
  ### Testing
  test_loss,test_acc=0,0
  model_0.eval()
  with torch.inference_mode():
    for X_test,y_test in test_dataloader: # test_dataloader에서는 왜 enumerate를 안할거지..? enumerate 가뭐지..-https://blog.naver.com/PostView.nhn?blogId=youndok&logNo=222053465832
    # test_dataloader===> (각 32장의 인풋 이미지 텐서들, 인풋이미지의 클래스 넘버 텐서)가 튜플형태로 반환
    # enumerate 를 안한 이유는 인덱싱번호가 생성이 되니까 그게아마 X_test에 반환이 되었을 거란 생각이든다. 그래서 model_0(X_test)에서 오류가 났을거란 생각이 듦. 그리고 y_test에는 (test_dataloader의 텐서와 텐서에 맞는 클래스 텐서의 튜플형태가 반환되었을것이다.)
      # 1. Forward pass
      test_pred=model_0(X_test)
      # 2. Calculate loss (accumulatively)
      test_loss += loss_fn(test_pred,y_test)
      # 3. Calculate acc
      test_acc+=accuracy_fn(y_true=y_test,y_pred=test_pred.argmax(dim=1))

    # Calculate the test loss average per batch
    test_loss /= len(test_dataloader)

    # Calculate the test acc average per batch
    test_acc/=len(test_dataloader)
  # Print out what's happening
  print(f"\n Train loss:{train_loss:.4f} | Test_loss: {test_loss:.4f}, Test acc: {test_acc:.4f}")
# Calculate training time
train_time_end_on_cpu=timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_cpu,
                                            end=train_time_end_on_cpu,
                                            device=str(next(model_0.parameters()).device))


In [ ]:
next(model_0.parameters()).device
type(model_0.parameters())

**Generator 는 이터레이터를 생성해주는 함수이다.**

-https://dojang.io/mod/page/view.php?id=2412

함수안에 yield 를 선언하면 generator 로 바뀐다




## 4. Make predictions and get model 0 results


In [ ]:
import torch
device="cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
torch.manual_seed(42)
def eval_model(model:torch.nn.Module,
               data_loader:torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn,
               device=device):
  """ Returns a dictionary containing the results of model predicting on data_loader."""
  loss, acc=0,0
  model.eval()
  with torch.inference_mode():
    for X,y in tqdm(data_loader):
      # Make our data device agnostic
      X,y=X.to(device),y.to(device)
      model=model.to(device)
      # Make predictions
      y_pred=model(X)
      # accumulate the loss and acc values per batch
      loss+=loss_fn(y_pred,y) #y_pred(logits)와 라벨링을 비교하여 loss 계산
      acc+=accuracy_fn(y_true=y,
                    y_pred=y_pred.argmax(dim=1))
      # Scale loss and acc to find the average loss/acc per batch
    loss /= len(data_loader)
    acc /=len(data_loader)
  return {"model_name": model.__class__.__name__, # only works when model was created with a class self.__class__.__name__를 참조하면 부모가 아닌 현재 클래스의 이름이 참조됩니다.
          "model_loss":loss.item(),
          "model_acc":acc}
# calculate model 0 results on test dataset.
model_0_results=eval_model(model=model_0,
                           data_loader=test_dataloader, # test_dataloader 는 인제 ..generator 로써 안에 32장 인풋 텐서의 1000/32개의 배치들이 존재.
                           loss_fn=loss_fn,
                           accuracy_fn=accuracy_fn
                           )
model_0_results
#print(len(X),len(test_dataloader),y)

## 5. Setup device agnostic-code (for using a GPU if there is one)

In [ ]:
# Setup device-agnostic code
import torch
device="cuda" if torch.cuda.is_available() else "cpu"
device

## 6. Model 1: Building a better model with non-linearity

We learned about the power of non-linearity in Notebook 02 -https://www.learnpytorch.io/02_pytorch_classification/#62-building-a-model-with-non-linearity

In [ ]:
# 내가만든 모델ㅋㅋㅋ
from torch import nn
class FashionMNISTModelV1(nn.Module):
  def __init__(self,
               input_shape:int,
               hidden_units:int,
               output_shape:int):
    super().__init__()

    self.layer_Flatten=nn.Flatten(),
    self.layer_Linear=nn.Linear(in_features=input_shape,
                  out_features=hidden_units),
    self.layer_Linear_last=nn.Linear(in_features=hidden_units,
                  out_features=output_shape)
    self.ReLU=nn.ReLU()
  def forward(self,x):
    return self.layer_Linear_last(self.ReLU(self.layer_Linear(self.ReLU(self.layer_Flatten(x)))))
model_1=FashionMNISTModelV1(input_shape=784,
                            hidden_units=10,
                            output_shape=len(train_data.classes)).to("cpu")
model_1,model_1.state_dict()

In [ ]:
# Create a model with non-linear and linear layers
class FashionMNISTModelV1(nn.Module):
  def __init__(self,
               input_shape:int,
               hidden_units:int,
               output_shape:int):
    super().__init__()
    self.layer_stack=nn.Sequential(
        nn.Flatten(), # Flatten inputs into a single vector.
        nn.Linear(in_features=input_shape,
                  out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units, # 이전 아웃피쳐와 동일
                  out_features=output_shape),
        nn.ReLU()
    )
  def forward(self,x:torch.Tensor):
    return self.layer_stack(x)

In [ ]:
next(model_0.parameters()).device

In [ ]:
# Create an instance of model_1
torch.manual_seed(42)
model_1=FashionMNISTModelV1(input_shape=784,
                            hidden_units=10,
                            output_shape=len(class_names)
                            ).to(device) # send to the GPU if it's available
next(model_1.parameters()).device


### 6.1 Set up loss, optimizer and evaluation metrics

In [ ]:
from helper_functions import accuracy_fn
loss_fn=nn.CrossEntropyLoss() # measure how wrong our model is
optimizer=torch.optim.SGD(params=model_1.parameters(), # tries to update our model's parameters to reduce the loss
                          lr=0.1)

### 6.2 Functionizing training and evaluation/testing loops

Let's create a function for:
* training loop - `train_step()`
* testing loop - `test_step()`

In [ ]:
def train_step(model:torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer,
               accuracy_fn,
               device:torch.device=device
               ):
  """ Perform a training with model trying to learn on data_loader."""
  train_loss,train_acc=0,0

  # put model into training mode
  model.train()

  # Add a loop to loop through the training batches
  for batch,(X,y) in enumerate(data_loader): # enumerate 원소랑 인덱싱 번호를 추출.
    # Put data on target device
    X,y=X.to(device), y.to(device)

    # 1. Forward pass
    y_pred=model(X) # outputs the raw logits from the model
    print(f"batch={batch}")

    # 2. Calculate loss / acc(per batch)
    loss=loss_fn(y_pred,y)
    train_loss+=loss # accumulate train loss
    train_acc+=accuracy_fn(y_true=y,
                           y_pred=y_pred.argmax(dim=1))

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. loss backward
    loss.backward() # 모델의 파라미터 기울기를 계산한다.

    # 5. Optimizer step
    optimizer.step() # 가중치 업데이트

 # Divide total train loss and acc by length of train dataloader
  train_loss /=len(data_loader)
  train_acc /=len(data_loader)
  print(f"Train Loss : {train_loss:.5f} | Train acc : {train_acc:.2f} %")


In [ ]:
### Testing
def test_step(model:torch.nn.Module,
              loss_fn:torch.nn.Module,
              test_dataloader:torch.utils.data.DataLoader,
              accuracy_fn,
              device:torch.device=device):
  """ Peforms a testing loop step on model going over data_loader."""
  test_loss,test_acc=0,0
  model.eval()
  with torch.inference_mode():
    for X_test,y_test in test_dataloader: # test_dataloader에서는 왜 enumerate를 안할거지..? enumerate 가뭐지..-https://blog.naver.com/PostView.nhn?blogId=youndok&logNo=222053465832
    # test_dataloader===> (각 32장의 인풋 이미지 텐서들, 인풋이미지의 클래스 넘버 텐서)가 튜플형태로 반환
    # enumerate 를 안한 이유는 인덱싱번호가 생성이 되니까 그게아마 X_test에 반환이 되었을 거란 생각이든다. 그래서 model_0(X_test)에서 오류가 났을거란 생각이 듦. 그리고 y_test에는 (test_dataloader의 텐서와 텐서에 맞는 클래스 텐서의 튜플형태가 반환되었을것이다.)
      X_test,y_test=X_test.to(device),y_test.to(device)
      # 1. Forward pass
      test_pred=model(X_test)
      # 2. Calculate loss (accumulatively)
      test_loss += loss_fn(test_pred,y_test)
      # 3. Calculate acc
      test_acc+=accuracy_fn(y_true=y_test,y_pred=test_pred.argmax(dim=1))

    # Calculate the test loss average per batch
    test_loss /= len(test_dataloader)

    # Calculate the test acc average per batch
    test_acc/=len(test_dataloader)
    print(f"Test loss: {test_loss:.5f} | Test acc: {test_acc:.2f} %")

In [ ]:
torch.manual_seed(42)

# CPU와 GPU의 시간 차이 측
# Measure time
from timeit import default_timer as timer
train_time_start_on_gpu=timer()

# Set epochs
epochs=3

# Create an optimization and evaluation loop using train_step() and test_step()
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch} \n-----------")
  train_step(model=model_1,
            data_loader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            accuracy_fn=accuracy_fn,
            device=device)
  test_step(model=model_1,
            test_dataloader= test_dataloader,
            loss_fn=loss_fn,
            accuracy_fn=accuracy_fn,
            device=device)

  train_time_end_on_gpu=timer()
  total_train_time_model_1=print_train_time(start=train_time_start_on_gpu,
                                            end=train_time_end_on_gpu,
                                            device=device)

 **Note:** sometimes, depending on your data / hardware you might find that your model trains faster on CPU than GPU

 why is this

 1. It could be that the overhead for copying data / model to and from the GPU outweighs the compute benefits offered by the GPU.
 2. The hardware you're using has a better CPU in terms compute capability than the GPU

 For more on how to make your models compute faster, see here:https://horace.io/brrr_intro.html


In [ ]:
# Get model_1 results dictionary
model_1_results=eval_model(model=model_1,
                          data_loader=test_dataloader,
                          loss_fn=loss_fn,
                          accuracy_fn=accuracy_fn)
model_1_results,

## Model 2: Building a Convolutional Neural Network(CNN)

CNN's are also known Convnets.
CNN's are known for their capabilities to find patterns in visual data.
To Find out what's happneing inside a CNN. https://poloclub.github.io/cnn-explainer/

In [ ]:
# Create a convolutional neural network
class FashionMNISTModelV2(nn.Module):
  """
  Model architecture that replicates the TinyVGG
  Model from CNN Explainer website.
  """
  def __init__(self, input_shape: int,hidden_units: int,output_shape:int):
    super().__init__()
    self.conv_block_1= nn.Sequential(
        # Create a conv layer - https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2,2))
    )
    self.conv_block_2=nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  padding=1,
                  stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.classifier=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*7*7,
                  out_features=output_shape)
    )
  def forward(self,x):
    x=self.conv_block_1(x)
    print(f"output shape of conv_block_1: {x.shape}")
    x=self.conv_block_2(x)
    print(f"output shape of conv_block_2: {x.shape}")
    x=self.classifier(x)
    print(f"output shape of classifier : {x.shape}")
    return x

In [ ]:
torch.manual_seed(42)
model_2=FashionMNISTModelV2(input_shape=1, # 컬러채널
                            hidden_units=10,
                            output_shape=len(class_names)
                            ).to(device)

In [ ]:
image.shape

In [ ]:
model_2.state_dict()

In [ ]:
plt.imshow(image.squeeze(),cmap='gray')

### 7.1 Stepping through `nn.Conv2d()`


In [ ]:
torch.manual_seed(42)
# Create a batch of images
images=torch.randn(size=(32,3,64,64))
test_image=images[0]
print(f"image batch shape:{images.shape}")
print(f"SIngle image shape:{test_image.shape}")
print(f"Test_image:\n {test_image}")

In [ ]:
# Create a single conv2d layer
conv_layer=nn.Conv2d(in_channels=3, # 인채널은 컬러 채널
                     out_channels=10,
                     kernel_size=3,
                     stride=1,
                     padding=0
                     )
# Pass the data through the convolutional layer
conv_output=conv_layer(test_image)
conv_output

### 7.2 Stepping through `nn.MaxPool2d()`
https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html

In [ ]:
test_image.shape

In [ ]:
# Print out original image shape without unsqueezed dimension
print(f"Test image original shape: {test_image.shape}")
print(f"test image with unsqueezed dimesion: {test_image.unsqueeze(0).shape}")

# Create a sample nn.MaxPool2d layer
max_pool_layer=nn.MaxPool2d(kernel_size=2)

# Pass data through just the conv_layer
test_image_through_conv=conv_layer(test_image.unsqueeze(dim=0))
print(f"shape after going through conv_layer():{test_image_through_conv.shape}")

# Pass data through the max pool layer
test_image_through_conv_and_max_pool=max_pool_layer(test_image_through_conv)
print(f"shape after going through conv_layer() and max_pool_layer(): {test_image_through_conv_and_max_pool.shape}")

# SHIFT ctrl SPACE - docstring

In [ ]:
torch.manual_seed(42)
# Create a random tensor with a similar number of dimensions to our images
random_tensor=torch.randn(size=(1,1,2,2))
print(f"Random tensor: \n {random_tensor}")
print(f"Randomw tensor shape: {random_tensor.shape}")
#Create maxpool layer
max_pool_layer=nn.MaxPool2d(kernel_size=2)

# Pass the random tensor through the maxpoollayer
max_pool_tensor=max_pool_layer(random_tensor)
print(f"\nMax pool tensor: \n{max_pool_tensor}")
print(f"Max pool tensor shape: \n{max_pool_tensor.shape}")

In [ ]:
rand_image_tensor=torch.randn(size=(1,28,28))
rand_image_tensor.shape

### 7.3 Setup Loss function and optimizer for model_2


In [ ]:
# Set up loss function/eval metrics/opitimizer
from helper_functions import accuracy_fn

loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(params=model_2.parameters(),
                          lr=0.1)


In [ ]:
model_2.state_dict()

### 7.4 Training and testing `model_2`using our training and test functions

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Measure time
from timeit import default_timer as timer
train_time_start_model_2 =timer()

# Train and test model
epochs=3
for epoch in tqdm(range(epochs)):
  print(f"Epoch : {epoch}\n-----")
  train_step(model=model_2,
             data_loader=train_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             accuracy_fn=accuracy_fn,
             device=device)
  test_step(model=model_2,
            test_dataloader= test_dataloader,
            loss_fn=loss_fn,
            accuracy_fn=accuracy_fn,
            device=device)

train_time_end_model_2=timer()
total_train_time_model_2=print_train_time(start=train_time_start_model_2,
                                          end=train_time_end_model_2,
                                          device=device)



In [ ]:
# Get model_2 results
model_2_results=eval_model(
    model=model_2,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn,
    device=device
)

model_2_results

## 8. Compare model results and trianing time

In [ ]:
import pandas as pd
compare_results=pd.DataFrame([model_0_results,
                              model_1_results,
                              model_2_results])
compare_results

In [ ]:
# Add training time to results comparision
compare_results["training_time"]=[total_train_time_model_0,
                                   total_train_time_model_1,
                                   total_train_time_model_2]

compare_results

In [ ]:
# Visualize our model results
compare_results.set_index("model_name")["model_acc"].plot(kind="barh")
plt.xlabel("accuracy (%)")
plt.ylabel("model")

## 9. Make and evaluate random predictions with best model

In [ ]:
def make_predictions(model:torch.nn.Module,
                      data:list,
                      device:torch.device=device):
  pred_probs=[]
  model.eval()
  model.to(device)
  with torch.inference_mode():
    for sample in data:
    # Prepare the sample (add a batch dimension and pass to target device)
      sample=torch.unsqueeze(sample,dim=0).to(device)

    # Forward pass (model outputs raw logits)
      pred_logit=model(sample)

    # Get prediction probability (logit->prediction probability)
      pred_prob=torch.softmax(pred_logit.squeeze(),dim=0)

    # Get pred_prob off the GPU for further calculation
      pred_probs.append(pred_prob.cpu())

  # Stack the pred_probs to turn list into a tensor
  return torch.stack(pred_probs)

In [ ]:
list(test_data)

In [ ]:
import random
# random.seed(42)
test_samples=[]
test_labels=[]

for sample,label in random.sample(list(test_data),k=9): #random.sample(list내부의 원소중 9개를 뽑)
  test_samples.append(sample)
  test_labels.append(label)
  # View the first sample shape
test_samples[0].shape

In [ ]:
plt.imshow(test_samples[0].squeeze(),cmap="gray")
plt.title(class_names[test_labels[0]])

In [ ]:
# Make predictions
pred_probs=make_predictions(model=model_2,
                            data=test_samples,
                            device=device)

# View first two prediction probabilties
pred_probs[:2]

In [ ]:
# Convert prediction probabilities to labels
pred_classes=pred_probs.argmax(dim=1)
pred_classes

In [ ]:
test_labels

In [ ]:
# Plot predictions
plt.figure(figsize=(9,9))
nrows=3
ncols=3
for i, sample in enumerate(test_samples):
  #Create subplot
  plt.subplot(nrows,ncols, i+1)
  print(i)
  # Plot the target image
  plt.imshow(sample.squeeze(),cmap="gray")

  # Find the prediction(in text form, e.g "Sandal")
  pred_label=class_names[pred_classes[i]] # pred_label에 예측한 클래스의 이름이 담김

  # Get the truth label(in text form)
  truth_label=class_names[test_labels[i]] # 정답값인 클래시 이름이 담김

  # Create a title for the plot
  title_text=f"pred:{pred_label} | Truth: {truth_label}"

  # Check for quality between pred and truth and change color of title text
  if pred_label==truth_label:
    plt.title(title_text,fontsize=10, c="g") # green text if prediction same as truth
  else:
    plt.title(title_text,fontsize=10, c="r") # 정답이 아니면 빨간

  plt.axis(False)

## Making a confusion matrix for further prediction evaluating

A Confusion matrix is a fantastic way of evaluating your classification models visually:
https://www.learnpytorch.io/02_pytorch_classification/#9-more-classification-evaluation-metrics

1. Make predictions with our trained model on the test datasets.
2.Make a confusion matrix `torchmetrics.ConfusionMatrix` -https://torchmetrics.readthedocs.io/en/stable/classification/confusion_matrix.html
3. Plot the confusion matrix using `mlxtend.plotting.plot_confusion_matrix()` -https://rasbt.github.io/mlxtend/user_guide/plotting/plot_confusion_matrix/

In [ ]:
# Import tqdm.auto
from tqdm.auto import tqdm

# 1. Make predictions with Trained model_2

y_preds=[]
model_2.eval()
with torch.inference_mode():
  for X,y in tqdm(test_dataloader, desc="Making predictions..."):
    # Send the data and targets to target device
    X,y=X.to(device),y.to(device)

    # Do the forward pass
    y_logits=model_2(X)

    # Turn predictions from logits -> Prediction probabilities -> prediction labels
    y_pred=torch.softmax(y_logits.squeeze(),dim=0).argmax(dim=1) #, dim=0은 열 방향으로 연산을 수행하고 dim=1은 행 방향으로 연산을 수행합니다.

    # Put prediction on CPU for evaluation
    y_preds.append(y_pred.cpu())

# Concatenate list of predictions into a tensor
print(y_preds)
y_preds_tensor=torch.cat(y_preds)
y_preds_tensor

In [ ]:
# See if required packages are installed and if not, install them...

try:
  import torchmetrics,mlxtend
  print(f"mlxtend version: {mlxtend.__version__}")
  assert int(mlxtend.__version__.split(".")[1]>=19,"mlxtend version should be 0.19.0 or higher")
except:
  !pip install -q torchmetrics -U mlxtend
  import torchmetrics,mlxtend
  print(f"mlxtend version:{mlxtend.__version__}")

In [ ]:
import mlxtend
mlxtend.__version__

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

# 2. Setup confusion instance and compare predictions to targets

confmat=ConfusionMatrix(task='multiclass',num_classes=len(class_names))
confmat_tensor=confmat(preds=y_preds_tensor,
                       target=test_data.targets)

# 3.plot the confusion matrix
fig , ax=plot_confusion_matrix(
    conf_mat=confmat_tensor.numpy(), # matplotlib likes working with numpy
    class_names=class_names,
    figsize=(10,7)
)

## 11. Save and load Best performing model

In [ ]:
from pathlib import Path

# Create model directory path
MODEL_PATH=Path("models")
MODEL_PATH.mkdir(parents=True,
                 exist_ok=True)

# Create model save
MODEL_NAME= "03_pytorch_computer_vision_model_2.pth"
MODEL_SAVE_PATH=MODEL_PATH/MODEL_NAME

# Save the model state_dict
print(f"Saving model to : {MODEL_SAVE_PATH}")
torch.save(obj=model_2.state_dict(),
           f=MODEL_SAVE_PATH)

In [ ]:
# Create a new instance

torch.manual_seed(42)

loaded_model_2=FashionMNISTModelV2(input_shape=1,
                                   hidden_units=10,
                                   output_shape=len(class_names))

# Load in the save state_dict()
loaded_model_2.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

# Send the model to the target device
loaded_model_2.to(device)

In [ ]:
model_2_results

In [ ]:
# Evaluate loaded model
torch.manual_seed(42)

loaded_model_2_results=eval_model(
    model=loaded_model_2,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn
)
loaded_model_2_results

In [ ]:
# Check if model results are close to each other
torch.isclose(torch.tensor(model_2_results["model_loss"]),
              torch.tensor(loaded_model_2_results["model_loss"]))